In [1]:
import matplotlib.pyplot as plt
import time
from IPython.display import clear_output
import random
from tqdm import tqdm

from environment import Environment
from reinforce.model import ReinforceModel

In [2]:
model = ReinforceModel(initial_population=1, state_size=20, action_size=5)

In [3]:
def train(num_episodes, max_steps):
    
    rewards = []
    success_count = 0
    
    max_food_collected = 0
    
    all_states = []
    for idx in tqdm(range(num_episodes)):
        episode_states = []
        env = Environment(rows=16, cols=16)

        i = 0
        lifetime_reward = 0

        couldnt_solve = False
        while (not env.is_done()):
            if i == max_steps:
                couldnt_solve = True
                break

            state = env.get_state()
            episode_states.append(state)
            action, _ = model.predict_action(0, state)

            reward = 0
            if action == 0:
                reward = env.move_up()
            elif action == 1:
                reward = env.move_down()
            elif action == 2:
                reward = env.move_left()
            elif action == 3:
                reward = env.move_right()
            elif action == 4:
                reward = env.ingest()
        
            lifetime_reward += reward
            
            
            model.update_reward(0, reward)
            
            i += 1
            
        all_states.append(episode_states)
            
#         clear_output(wait=True)
#         print(f"Episode: [{idx+1}/{num_episodes}]")
#         print(f"Food collected: {env.consumed_count}")
        
        if env.consumed_count > max_food_collected:
            max_food_collected = env.consumed_count

        model.update_all_agents(0)
        
        rewards.append(lifetime_reward)
        
    print(f"Max food collected: {max_food_collected}")
        
    return rewards, all_states

In [4]:
num_episodes=100000
max_steps = 250

rewards, all_states = train(num_episodes=num_episodes, max_steps=max_steps)

  0%|          | 89/100000 [00:13<4:14:12,  6.55it/s]


KeyboardInterrupt: 

In [ ]:
import torch

torch.save(model.agents[0].state_dict(), f'experiment-9-{num_episodes//1000}k.pth')